In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
pd.set_option('display.max_columns',1000)
pd.set_option('display.max_rows',1000)

In [2]:
df = pd.read_csv('df_bp5.csv')

C:\Users\sanch\AppData\Local\Temp\ipykernel_24172\4107656890.py:1: DtypeWarning: Columns (13,14,15,19,85,87,198) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('df_bp5.csv')


In [3]:
df

,date,dblheader_code,day_of_week,team_v,league_v,game_no_v,team_h,league_h,game_no_h,runs_v,runs_h,outs_total,day_night,completion_info,forfeit_info,protest_info,ballpark_id,attendance,game_minutes,linescore_v,linescore_h,AB_v,H_v,2B_v,3B_v,HR_v,RBI_v,SH_v,SF_v,HBP_v,BB_v,IBB_v,SO_v,SB_v,CS_v,GIDP_v,CI_v,LOB_v,P_num_v,ERind_v,ERteam_v,WP_v,balk_v,PO_v,ASST_v,ERR_v,PB_v,DP_v,TP_v,AB_h,H_h,2B_h,3B_h,HR_h,RBI_h,SH_h,SF_h,HBP_h,BB_h,IBB_h,SO_h,SB_h,CS_h,GIDP_h,CI_h,LOB_h,P_num_h,ERind_h,ERteam_h,WP_h,balk_h,PO_h,ASST_h,ERR_h,PB_h,DP_h,TP_h,ump_HB_id,ump_HB_name,ump_1B_id,ump_1B_name,ump_2B_id,ump_2B_name,ump_3B_id,ump_3B_name,ump_LF_id,ump_LF_name,ump_RF_id,ump_RF_name,mgr_id_v,mgr_name_v,mgr_id_h,mgr_name_h,pitcher_id_w,pitcher_name_w,pitcher_id_l,pitcher_name_l,pitcher_id_s,pitcher_name_s,GWRBI_id,GWRBI_name,pitcher_start_id_v,pitcher_start_name_v,pitcher_start_id_h,pitcher_start_name_h,batter1_name_v,batter1_id_v,batter1_pos_v,batter2_name_v,batter2_id_v,batter2_pos_v,batter3_name_v,batter3_id_v,batter3_pos_v,batter4_name_v,batter4_id_v,batter4_pos_v,batter5_name_v,batter5_id_v,batter5_pos_v,batter6_name_v,batter6_id_v,batter6_pos_v,batter7_name_v,batter7_id_v,batter7_pos_v,batter8_name_v,batter8_id_v,batter8_pos_v,batter9_name_v,batter9_id_v,batter9_pos_v,batter1_name_h,batter1_id_h,batter1_pos_h,batter2_name_h,batter2_id_h,batter2_pos_h,batter3_name_h,batter3_id_h,batter3_pos_h,batter4_name_h,batter4_id_h,batter4_pos_h,batter5_name_h,batter5_id_h,batter5_pos_h,batter6_name_h,batter6_id_h,batter6_pos_h,batter7_name_h,batter7_id_h,batter7_pos_h,batter8_name_h,batter8_id_h,batter8_pos_h,batter9_name_h,batter9_id_h,batter9_pos_h,misc_info,acqui_info,season,run_diff,home_victory,run_total,date_dblhead,BATAVG_162_h,BATAVG_162_v,OBP_162_h,OBP_162_v,SLG_162_h,SLG_162_v,OBS_162_h,OBS_162_v,SB_162_h,SB_162_v,CS_162_h,CS_162_v,ERR_162_h,ERR_162_v,BATAVG_30_h,BATAVG_30_v,OBP_30_h,OBP_30_v,SLG_30_h,SLG_30_v,OBS_30_h,OBS_30_v,SB_30_h,SB_30_v,CS_30_h,CS_30_v,ERR_30_h,ERR_30_v,implied_prob_h,implied_prob_v,implied_prob_h_mid,over_under_line,over_under_result,IP_real_h,IP_real_v,rollsum_IP_real_10_h,rollsum_IP_real_10_v,rollsum_H_10_h,rollsum_H_10_v,rollsum_BFP_10_h,rollsum_BFP_10_v,rollsum_HR_10_h,rollsum_HR_10_v,rollsum_R_10_h,rollsum_R_10_v,rollsum_ER_10_h,rollsum_ER_10_v,rollsum_BB_10_h,rollsum_BB_10_v,rollsum_IB_10_h,rollsum_IB_10_v,rollsum_SO_10_h,rollsum_SO_10_v,rollsum_SH_10_h,rollsum_SH_10_v,rollsum_SF_10_h,rollsum_SF_10_v,rollsum_WP_10_h,rollsum_WP_10_v,rollsum_HBP_10_h,rollsum_HBP_10_v,rollsum_BK_10_h,rollsum_BK_10_v,rollsum_2B_10_h,rollsum_2B_10_v,rollsum_3B_10_h,rollsum_3B_10_v,rollsum_IP_real_35_h,rollsum_IP_real_35_v,rollsum_H_35_h,rollsum_H_35_v,rollsum_BFP_35_h,rollsum_BFP_35_v,rollsum_HR_35_h,rollsum_HR_35_v,rollsum_R_35_h,rollsum_R_35_v,rollsum_ER_35_h,rollsum_ER_35_v,rollsum_BB_35_h,rollsum_BB_35_v,rollsum_IB_35_h,rollsum_IB_35_v,rollsum_SO_35_h,rollsum_SO_35_v,rollsum_SH_35_h,rollsum_SH_35_v,rollsum_SF_35_h,rollsum_SF_35_v,rollsum_WP_35_h,rollsum_WP_35_v,rollsum_HBP_35_h,rollsum_HBP_35_v,rollsum_BK_35_h,rollsum_BK_35_v,rollsum_2B_35_h,rollsum_2B_35_v,rollsum_3B_35_h,rollsum_3B_35_v,H_BB_roll_10_h,H_BB_roll_10_v,XB_roll_10_h,XB_roll_10_v,TB_roll_10_h,TB_roll_10_v,IP_mod_10_h,IP_mod_10_v,BF_mod_10_h,BF_mod_10_v,ER_mod_10_h,ER_mod_10_v,H_BB_mod_10_h,H_BB_mod_10_v,H_BB_perc_10_h,H_BB_perc_10_v,SO_mod_10_h,SO_mod_10_v,TB_BB_mod_10_h,TB_BB_mod_10_v,ERA_10_h,ERA_10_v,WHIP_10_h,WHIP_10_v,SO_perc_10_h,SO_perc_10_v,TB_BB_perc_10_h,TB_BB_perc_10_v,H_BB_roll_35_h,H_BB_roll_35_v,XB_roll_35_h,XB_roll_35_v,TB_roll_35_h,TB_roll_35_v,IP_mod_35_h,IP_mod_35_v,BF_mod_35_h,BF_mod_35_v,ER_mod_35_h,ER_mod_35_v,H_BB_mod_35_h,H_BB_mod_35_v,H_BB_perc_35_h,H_BB_perc_35_v,SO_mod_35_h,SO_mod_35_v,TB_BB_mod_35_h,TB_BB_mod_35_v,ERA_35_h,ERA_35_v,WHIP_35_h,WHIP_35_v,SO_perc_35_h,SO_perc_35_v,TB_BB_perc_35_h,TB_BB_perc_35_v
0,19940403,0,Sun,SLN,NL,1,CIN,NL,1,6,4,54,N,NaN,NaN,NaN,CIN08,32803.0,169,102210000,030001000,35,11,2,0,1,5,2,0,0,4,1,8,1,1,0,0,8,3,

In [4]:
df.shape

(67335, 321)

In [5]:
df = df.dropna(subset=['OBP_162_h', 'OBP_162_v', 'SLG_162_h', 'SLG_162_v'])

In [6]:
df_train = df[(df.season>1995) & (df.season<=2020)]
df_test = df[df.season>=2021]

In [7]:
X = ['ERA_10_h', 'ERA_10_v',
     'ERA_35_h', 'ERA_35_v',
     'OBP_162_h', 'OBP_162_v',
     'OBP_30_h', 'OBP_30_v',
     'SLG_162_h', 'SLG_162_v',
     'SLG_30_h', 'SLG_30_v',
     'SO_perc_10_h', 'SO_perc_10_v',
     'SO_perc_35_h', 'SO_perc_35_v',
     'WHIP_10_h', 'WHIP_10_v',
     'WHIP_35_h', 'WHIP_35_v']
     
target = 'home_victory'


In [8]:
X_train = df_train.loc[:,X]
X_test = df_test.loc[:,X]

y_train = df_train[target].to_numpy()
y_test = df_test[target].to_numpy()

In [9]:
X_train.isna().sum()

ERA_10_h        0
ERA_10_v        0
ERA_35_h        0
ERA_35_v        0
OBP_162_h       0
OBP_162_v       0
OBP_30_h        0
OBP_30_v        0
SLG_162_h       0
SLG_162_v       0
SLG_30_h        0
SLG_30_v        0
SO_perc_10_h    0
SO_perc_10_v    0
SO_perc_35_h    0
SO_perc_35_v    0
WHIP_10_h       0
WHIP_10_v       0
WHIP_35_h       0
WHIP_35_v       0
dtype: int64

In [10]:
smote = SMOTE()

In [11]:
X_train_sm, y_train_sm = smote.fit_resample(X_train.to_numpy(), y_train)

In [12]:
"""from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# Definir los parámetros que deseas probar
param_grid = {
    'n_estimators': [10, 15, 20, 30],
    'max_depth': [3, 5, 7, 9, None],
    'criterion': ['gini', 'entropy', 'log_loss'],
    'n_jobs': [4]
}

# Crear el modelo de Random Forest
forest = RandomForestClassifier()

# Realizar la búsqueda aleatoria
random_search = RandomizedSearchCV(forest, param_distributions=param_grid, n_iter=10, cv=5)
random_search.fit(X_train_sm, y_train_sm)
#forest.score(X_test.to_numpy(),y_test)

# Imprimir los mejores parámetros y el mejor score
print("Mejores parámetros:", random_search.best_params_)
print("Mejor score:", random_search.best_score_)"""

'from sklearn.ensemble import RandomForestClassifier\nfrom sklearn.model_selection import RandomizedSearchCV\nimport numpy as np\n\n# Definir los parámetros que deseas probar\nparam_grid = {\n    \'n_estimators\': [10, 15, 20, 30],\n    \'max_depth\': [3, 5, 7, 9, None],\n    \'criterion\': [\'gini\', \'entropy\', \'log_loss\'],\n    \'n_jobs\': [4]\n}\n\n# Crear el modelo de Random Forest\nforest = RandomForestClassifier()\n\n# Realizar la búsqueda aleatoria\nrandom_search = RandomizedSearchCV(forest, param_distributions=param_grid, n_iter=10, cv=5)\nrandom_search.fit(X_train_sm, y_train_sm)\n#forest.score(X_test.to_numpy(),y_test)\n\n# Imprimir los mejores parámetros y el mejor score\nprint("Mejores parámetros:", random_search.best_params_)\nprint("Mejor score:", random_search.best_score_)'

In [13]:
forest = RandomForestClassifier(n_estimators=10, max_depth=None, criterion='log_loss', n_jobs=4)

In [14]:
forest.fit(X_train_sm, y_train_sm)

RandomForestClassifier(criterion='log_loss', n_estimators=10, n_jobs=4)

In [15]:
forest.score(X_train_sm,y_train_sm)

0.9856827489122089

In [16]:
forest.score(X_test.to_numpy(),y_test)

0.5274113767518549

In [17]:
model = LogisticRegression()
model.fit(X_train, y_train)

C:\Users\sanch\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [18]:
y_pred = model.predict(X_test)

In [19]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.5882110469909316


### Ponerlo para datos reales de hoy

In [20]:
columnas = ['Home_Team','Away_Team','ERA_10_h', 'ERA_10_v', 'ERA_35_h', 'ERA_35_v', 'OBP_162_h', 'OBP_162_v', 'OBP_30_h', 'OBP_30_v', 'SLG_162_h', 'SLG_162_v', 'SLG_30_h', 'SLG_30_v', 'SO_perc_10_h', 'SO_perc_10_v', 'SO_perc_35_h', 'SO_perc_35_v', 'WHIP_10_h', 'WHIP_10_v', 'WHIP_35_h', 'WHIP_35_v']

Partidos_hoy = pd.DataFrame(columns=columnas)

In [21]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [22]:
url = "https://www.baseball-reference.com/previews/"

In [23]:
response = requests.get(url)
response

<Response [200]>

In [24]:
response = requests.get(url)

In [25]:
soup = BeautifulSoup(response.content, "html.parser")

In [26]:
Away_Teams = []
Home_Teams = []
Away_Pitcher = []
Home_Pitcher = []
for i in range(0,len(soup.select("table td a")),5):
    print(soup.select("table td a")[i].get_text())
    Away_Teams.append(soup.select("table td a")[i].get_text())

D'backs
Yankees
Brewers
Cubs
Tigers
Orioles
Guardians
Mariners
Rockies
Royals
Athletics
Dodgers
Twins
Red Sox
Marlins


In [27]:
for i in range(2,len(soup.select("table td a")),5):
    print(soup.select("table td a")[i].get_text())
    Home_Teams.append(soup.select("table td a")[i].get_text())

Pirates
Reds
Rays
Phillies
Nationals
Blue Jays
Mets
Braves
Rangers
White Sox
Astros
Cardinals
Angels
Padres
Giants


In [28]:
for i in range(3,len(soup.select("table td a")),5):
    print(soup.select("table td a")[i].get_text())
    Away_Pitcher.append(soup.select("table td a")[i].get_text())

Zac Gallen
Clarke Schmidt
Adrian Houser
Marcus Stroman
Matthew Boyd
Kyle Gibson
Cal Quantrill
Bryce Miller
Karl Kauffmann
Zack Greinke
Ken Waldichuk
Tony Gonsolin
Joe Ryan
James Paxton
Sandy Alcantara


In [29]:
for i in range(4,len(soup.select("table td a")),5):
    print(soup.select("table td a")[i].get_text())
    Home_Pitcher.append(soup.select("table td a")[i].get_text())

Johan Oviedo
Ben Lively
Shane McClanahan
Ranger Suárez
Jake Irvin
Yusei Kikuchi
Carlos Carrasco
Bryce Elder
Martín Pérez
Michael Kopech
Brandon Bielak
Steven Matz
Reid Detmers
Blake Snell
Anthony DeSclafani


In [30]:
Todays_Games = { 'Away_Team': Away_Teams,
                'Home_Team': Home_Teams,
                'Away_Pitcher': Away_Pitcher,
                'Home_Pitcher': Home_Pitcher
    
}

In [31]:
Todays_Games = pd.DataFrame(Todays_Games)
Todays_Games

,Away_Team,Home_Team,Away_Pitcher,Home_Pitcher
0,D'backs,Pirates,Zac Gallen,Johan Oviedo
1,Yankees,Reds,Clarke Schmidt,Ben Lively
2,Brewers,Rays,Adrian Houser,Shane McClanahan
3,Cubs,Phillies,Marcus Stroman,Ranger Suárez
4,Tigers,Nationals,Matthew Boyd,Jake Irvin
5,Orioles,Blue Jays,Kyle Gibson,Yusei Kikuchi
6,Guardians,Mets,Cal Quantrill,Carlos Carrasco
7,Mariners,Braves,Bryce Miller,Bryce Elder
8,Rockies,Rangers,Karl Kauffmann,Martín Pérez
9,Royals,White Sox,Zack Greinke,Michael Kopech


In [32]:
from unidecode import unidecode

In [33]:
Todays_Games['Away_Pitcher'] = Todays_Games['Away_Pitcher'].apply(lambda x: unidecode(x) if isinstance(x, str) else x)

In [34]:
Todays_Games['Home_Pitcher'] = Todays_Games['Home_Pitcher'].apply(lambda x: unidecode(x) if isinstance(x, str) else x)

In [35]:
for index, row in Todays_Games.iterrows():
    nombre_pitcher = row['Away_Pitcher']
    id_pitcher = df.loc[df['pitcher_start_name_v'] == nombre_pitcher, 'pitcher_start_id_v'].values
    if len(id_pitcher) > 0:
        Todays_Games.at[index, 'Away_Pitcher_ID'] = id_pitcher[0]

In [36]:
for index, row in Todays_Games.iterrows():
    nombre_pitcher = row['Home_Pitcher']
    id_pitcher = df.loc[df['pitcher_start_name_h'] == nombre_pitcher, 'pitcher_start_id_h'].values
    if len(id_pitcher) > 0:
        Todays_Games.at[index, 'Home_Pitcher_ID'] = id_pitcher[0]

In [37]:
Todays_Games

,Away_Team,Home_Team,Away_Pitcher,Home_Pitcher,Away_Pitcher_ID,Home_Pitcher_ID
0,D'backs,Pirates,Zac Gallen,Johan Oviedo,gallz001,oviej001
1,Yankees,Reds,Clarke Schmidt,Ben Lively,schmc002,liveb001
2,Brewers,Rays,Adrian Houser,Shane McClanahan,housa001,mccls003
3,Cubs,Phillies,Marcus Stroman,Ranger Suarez,strom001,suarr001
4,Tigers,Nationals,Matthew Boyd,Jake Irvin,NaN,NaN
5,Orioles,Blue Jays,Kyle Gibson,Yusei Kikuchi,gibsk002,kikuy001
6,Guardians,Mets,Cal Quantrill,Carlos Carrasco,quanc001,carrc003
7,Mariners,Braves,Bryce Miller,Bryce Elder,NaN,eldeb001
8,Rockies,Rangers,Karl Kauffmann,Martin Perez,NaN,perem004
9,Royals,White Sox,Zack Greinke,Michael Kopech,greiz001,kopem001


In [38]:
for index, row in Todays_Games.iterrows():
    id_home = row['Away_Pitcher_ID']
    ERA_10 = df.loc[df['pitcher_start_id_v'] == id_home, 'ERA_10_h'].values
    if len(ERA_10) > 0:
        Todays_Games.at[index,'ERA_10_h'] = ERA_10[-1]

In [39]:
Todays_Games

,Away_Team,Home_Team,Away_Pitcher,Home_Pitcher,Away_Pitcher_ID,Home_Pitcher_ID,ERA_10_h
0,D'backs,Pirates,Zac Gallen,Johan Oviedo,gallz001,oviej001,3.807692
1,Yankees,Reds,Clarke Schmidt,Ben Lively,schmc002,liveb001,4.602273
2,Brewers,Rays,Adrian Houser,Shane McClanahan,housa001,mccls003,5.744681
3,Cubs,Phillies,Marcus Stroman,Ranger Suarez,strom001,suarr001,4.066265
4,Tigers,Nationals,Matthew Boyd,Jake Irvin,NaN,NaN,NaN
5,Orioles,Blue Jays,Kyle Gibson,Yusei Kikuchi,gibsk002,kikuy001,2.958904
6,Guardians,Mets,Cal Quantrill,Carlos Carrasco,quanc001,carrc003,4.235294
7,Mariners,Braves,Bryce Miller,Bryce Elder,NaN,eldeb001,NaN
8,Rockies,Rangers,Karl Kauffmann,Martin Perez,NaN,perem004,NaN
9,Royals,White Sox,Zack Greinke,Michael Kopech,greiz001,kopem001,2.306533


In [40]:
def Agregar_ultimos_datos_pitcher_home(columna):
       for index, row in Todays_Games.iterrows():
        id_home = row['Home_Pitcher_ID']
        variable = df.loc[df['pitcher_start_id_h'] == id_home, columna].values
        if len(variable) > 0:
            Todays_Games.at[index,columna] = variable[-1] 

In [41]:
def Agregar_ultimos_datos_pitcher_visit(columna):
       for index, row in Todays_Games.iterrows():
        id_away = row['Away_Pitcher_ID']
        variable = df.loc[df['pitcher_start_id_v'] == id_away, columna].values
        if len(variable) > 0:
            Todays_Games.at[index,columna] = variable[-1] 

In [42]:
df['team_v'].unique()

array(['SFN', 'MON', 'ATL', 'FLO', 'KCA', 'TEX', 'MIN', 'HOU', 'PHI',
       'LAN', 'CHN', 'BOS', 'MIL', 'COL', 'CLE', 'TOR', 'CAL', 'OAK',
       'NYN', 'PIT', 'BAL', 'CHA', 'DET', 'SLN', 'SDN', 'CIN', 'NYA',
       'SEA', 'ANA', 'TBA', 'ARI', 'WAS', 'MIA'], dtype=object)

In [43]:
def Agregar_ultimos_datos_bat_home(columna):
       for index, row in Todays_Games.iterrows():
        id_home = row['Home_Team_short']
        #print(id_home)
        variable = df.loc[df['team_h'] == id_home, columna].values
        if len(variable) > 0:
            Todays_Games.at[index,columna] = variable[-1] 

In [44]:
def Agregar_ultimos_datos_bat_away(columna):
       for index, row in Todays_Games.iterrows():
        id_visit = row['Away_Team_short']
        #print(id_home)
        variable = df.loc[df['team_v'] == id_visit, columna].values
        if len(variable) > 0:
            Todays_Games.at[index,columna] = variable[-1] 

In [45]:
columnas = ['ERA_10_h', 'ERA_10_v', 'ERA_35_h', 'ERA_35_v', 'OBP_162_h', 'OBP_162_v', 'OBP_30_h', 'OBP_30_v', 'SLG_162_h', 'SLG_162_v', 'SLG_30_h', 'SLG_30_v', 'SO_perc_10_h', 'SO_perc_10_v', 'SO_perc_35_h', 'SO_perc_35_v', 'WHIP_10_h', 'WHIP_10_v', 'WHIP_35_h', 'WHIP_35_v']
for columna in columnas:
    Todays_Games[columna] = ''

In [46]:
Lista = Todays_Games.columns

In [47]:
Lista

Index(['Away_Team', 'Home_Team', 'Away_Pitcher', 'Home_Pitcher',
       'Away_Pitcher_ID', 'Home_Pitcher_ID', 'ERA_10_h', 'ERA_10_v',
       'ERA_35_h', 'ERA_35_v', 'OBP_162_h', 'OBP_162_v', 'OBP_30_h',
       'OBP_30_v', 'SLG_162_h', 'SLG_162_v', 'SLG_30_h', 'SLG_30_v',
       'SO_perc_10_h', 'SO_perc_10_v', 'SO_perc_35_h', 'SO_perc_35_v',
       'WHIP_10_h', 'WHIP_10_v', 'WHIP_35_h', 'WHIP_35_v'],
      dtype='object')

In [48]:
datos_pitch_v = [titulo for titulo in Lista if titulo.endswith("_v")]
datos_pitch_v.remove('OBP_162_v')
datos_pitch_v.remove('OBP_30_v')
datos_pitch_v.remove('SLG_162_v')
datos_pitch_v.remove('SLG_30_v')

In [49]:
datos_pitch_h = [titulo for titulo in Lista if titulo.endswith("_h")]
datos_pitch_h.remove('OBP_162_h')
datos_pitch_h.remove('OBP_30_h')
datos_pitch_h.remove('SLG_162_h')
datos_pitch_h.remove('SLG_30_h')


In [50]:
datos_bat_h = ['OBP_162_h','OBP_30_h','SLG_162_h','SLG_30_h']

In [51]:
datos_bat_v = ['OBP_162_v','OBP_30_v','SLG_162_v','SLG_30_v']

In [52]:
mapeo_siglas = {
    'Red Sox': 'BOS',
    'Yankees': 'NYA',
    'Dodgers': 'LAD',
    'Cubs': 'CHC',
    'Reds': 'CIN',
    'Orioles': 'BAL',
    'Guardians':'CLE',
    'Blue Jays': 'TOR',
    'Rockies': 'COL',
    'Tigers': 'DET',
    'Marlins': 'MIA',
    'Mets': 'NYN',
    'Cardinals': 'SLN',
    'Rangers': 'TEX',
    'White Sox': 'CHA',
    'Astros': 'HOU',
    'Athletics': 'OAK',
    'Padres': 'SDN',
    'Giants': 'SFN',
    'Dodgers':'LAN',
    'Angels' :'ANA',
    'Pirates': 'PIT',
    'Nationals': 'WAS',
    'Mariners': 'SEA',
    'Rays': 'TBA',
    'Brewers':'MIL',
    'Braves': 'ATL',
    "D'backs" : 'ARI',
    'Royals': 'KCA',
    'Phillies': 'PHI',
    'Twins': 'MIN'
}

In [53]:
Todays_Games.insert(1, 'Away_Team_short', Todays_Games['Away_Team'].map(mapeo_siglas))

In [54]:
Todays_Games.insert(3, 'Home_Team_short', Todays_Games['Home_Team'].map(mapeo_siglas))

In [55]:
Todays_Games

,Away_Team,Away_Team_short,Home_Team,Home_Team_short,Away_Pitcher,Home_Pitcher,Away_Pitcher_ID,Home_Pitcher_ID,ERA_10_h,ERA_10_v,ERA_35_h,ERA_35_v,OBP_162_h,OBP_162_v,OBP_30_h,OBP_30_v,SLG_162_h,SLG_162_v,SLG_30_h,SLG_30_v,SO_perc_10_h,SO_perc_10_v,SO_perc_35_h,SO_perc_35_v,WHIP_10_h,WHIP_10_v,WHIP_35_h,WHIP_35_v
0,D'backs,ARI,Pirates,PIT,Zac Gallen,Johan Oviedo,gallz001,oviej001,,,,,,,,,,,,,,,,,,,,
1,Yankees,NYA,Reds,CIN,Clarke Schmidt,Ben Lively,schmc002,liveb001,,,,,,,,,,,,,,,,,,,,
2,Brewers,MIL,Rays,TBA,Adrian Houser,Shane McClanahan,housa001,mccls003,,,,,,,,,,,,,,,,,,,,
3,Cubs,CHC,Phillies,PHI,Marcus Stroman,Ranger Suarez,strom001,suarr001,,,,,,,,,,,,,,,,,,,,
4,Tigers,DET,Nationals,WAS,Matthew Boyd,Jake Irvin,NaN,NaN,,,,,,,,,,,,,,,,,,,,
5,Orioles,BAL,Blue Jays,TOR,Kyle Gibson,Yusei Kikuchi,gibsk002,kikuy001,,,,,,,,,,,,,,,,,,,,
6,Guardians,CLE,Mets,NYN,Cal Quantrill,Carlos Carrasco,quanc001,carrc003,,,,,,,,,,,,,,,,,,,,
7,Mariners,SEA,Braves,ATL,Bryce Miller,Bryce Elder,NaN,eldeb001,,,,,,,,,,,,,,,,,,,,
8,Rockies,COL,Rangers,TEX,Karl Kauffmann,Martin Perez,NaN,perem004,,,,,,,,,,,,,,,,,,,,
9,Royals,KCA,White Sox,CHA,Zack Greinke,Michael Kopech,greiz001,kopem001,,,,,,,,,,,,,,,,,,,,


In [56]:
for i in datos_pitch_v:
    Agregar_ultimos_datos_pitcher_visit(i)

In [57]:
for i in datos_pitch_h:
    Agregar_ultimos_datos_pitcher_home(i)

In [58]:
for i in datos_bat_h:
    Agregar_ultimos_datos_bat_home(i)

In [59]:
for i in datos_bat_v:
    Agregar_ultimos_datos_bat_away(i)

In [60]:
columns_use_predict = ['ERA_10_h', 'ERA_10_v', 'ERA_35_h', 'ERA_35_v', 'OBP_162_h',
       'OBP_162_v', 'OBP_30_h', 'OBP_30_v', 'SLG_162_h', 'SLG_162_v',
       'SLG_30_h', 'SLG_30_v', 'SO_perc_10_h', 'SO_perc_10_v', 'SO_perc_35_h',
       'SO_perc_35_v', 'WHIP_10_h', 'WHIP_10_v', 'WHIP_35_h', 'WHIP_35_v']

In [61]:
for i in columns_use_predict:
    Todays_Games.replace('', np.nan, inplace=True)
    Todays_Games[i] = Todays_Games[i].astype(float)

In [62]:
Todays_Games

,Away_Team,Away_Team_short,Home_Team,Home_Team_short,Away_Pitcher,Home_Pitcher,Away_Pitcher_ID,Home_Pitcher_ID,ERA_10_h,ERA_10_v,ERA_35_h,ERA_35_v,OBP_162_h,OBP_162_v,OBP_30_h,OBP_30_v,SLG_162_h,SLG_162_v,SLG_30_h,SLG_30_v,SO_perc_10_h,SO_perc_10_v,SO_perc_35_h,SO_perc_35_v,WHIP_10_h,WHIP_10_v,WHIP_35_h,WHIP_35_v
0,D'backs,ARI,Pirates,PIT,Zac Gallen,Johan Oviedo,gallz001,oviej001,3.126316,1.064039,4.286908,2.712919,0.285542,0.299966,0.301457,0.278704,0.363500,0.384069,0.376376,0.366633,0.240876,0.337553,0.202226,0.268054,1.263158,0.650246,1.454039,0.937799
1,Yankees,NYA,Reds,CIN,Clarke Schmidt,Ben Lively,schmc002,liveb001,4.313609,4.222222,4.336364,4.419048,0.294582,0.318867,0.268501,0.327679,0.374025,0.425724,0.338477,0.435259,0.181070,0.235000,0.155650,0.208095,1.295858,1.566667,1.336364,1.528571
2,Brewers,MIL,Rays,TBA,Adrian Houser,Shane McClanahan,housa001,mccls003,3.413793,6.496241,2.486842,3.820268,0.309027,0.308988,0.314932,0.318505,0.388920,0.410223,0.388177,0.424631,0.248908,0.158416,0.303413,0.166887,1.034483,1.669173,0.976974,1.388145
3,Cubs,CHC,Phillies,PHI,Marcus Stroman,Ranger Suarez,strom001,suarr001,2.715976,3.103448,2.984694,3.600000,0.312689,NaN,0.328182,NaN,0.422628,NaN,0.441468,NaN,0.216450,0.165975,0.213497,0.214018,1.136095,1.206897,1.250000,1.164103
4,Tigers,DET,Nationals,WAS,Matthew Boyd,Jake Irvin,NaN,NaN,NaN,NaN,NaN,NaN,0.307367,0.281646,0.311151,0.289788,0.378215,0.347341,0.401157,0.394685,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Orioles,BAL,Blue Jays,TOR,Kyle Gibson,Yusei Kikuchi,gibsk002,kikuy001,6.929204,5.333333,5.272532,5.017699,0.324752,0.297174,0.338287,0.303442,0.436556,0.391512,0.452975,0.407115,0.255814,0.221757,0.243553,0.213135,1.646018,1.518519,1.551502,1.322124
6,Guardians,CLE,Mets,NYN,Cal Quantrill,Carlos Carrasco,quanc001,carrc003,3.306122,2.777143,4.127698,3.353503,0.320874,0.307641,0.337168,0.307965,0.409555,0.383105,0.428431,0.369275,0.236715,0.193133,0.229695,0.171893,1.306122,1.080000,1.327338,1.208599
7,Mariners,SEA,Braves,ATL,Bryce Miller,Bryce Elder,NaN,eldeb001,3.669903,NaN,4.565079,NaN,0.310981,0.304428,0.306481,0.290055,0.444929,0.390407,0.442557,0.410256,0.202703,NaN,0.200952,NaN,1.368932,NaN,1.457143,NaN
8,Rockies,COL,Rangers,TEX,Karl Kauffmann,Martin Perez,NaN,perem004,3.943820,NaN,2.933786,NaN,0.295903,0.311385,0.262559,0.290528,0.392733,0.398666,0.367781,0.388503,0.195402,NaN,0.206813,NaN,1.432584,NaN,1.242784,NaN
9,Royals,KCA,White Sox,CHA,Zack Greinke,Michael Kopech,greiz001,kopem001,4.661871,2.531250,3.545455,4.268382,0.304283,0.303559,0.279197,0.303196,0.387344,0.379114,0.391514,0.380435,0.191176,0.132159,0.243590,0.132732,1.489209,1.368750,1.189394,1.329044


In [63]:
Todays_Games_clean = Todays_Games.dropna()

In [64]:
Todays_Games_to_predict = Todays_Games_clean[columns_use_predict].copy()

In [65]:
Todays_Games_to_predict

,ERA_10_h,ERA_10_v,ERA_35_h,ERA_35_v,OBP_162_h,OBP_162_v,OBP_30_h,OBP_30_v,SLG_162_h,SLG_162_v,SLG_30_h,SLG_30_v,SO_perc_10_h,SO_perc_10_v,SO_perc_35_h,SO_perc_35_v,WHIP_10_h,WHIP_10_v,WHIP_35_h,WHIP_35_v
0,3.126316,1.064039,4.286908,2.712919,0.285542,0.299966,0.301457,0.278704,0.363500,0.384069,0.376376,0.366633,0.240876,0.337553,0.202226,0.268054,1.263158,0.650246,1.454039,0.937799
1,4.313609,4.222222,4.336364,4.419048,0.294582,0.318867,0.268501,0.327679,0.374025,0.425724,0.338477,0.435259,0.181070,0.235000,0.155650,0.208095,1.295858,1.566667,1.336364,1.528571
2,3.413793,6.496241,2.486842,3.820268,0.309027,0.308988,0.314932,0.318505,0.388920,0.410223,0.388177,0.424631,0.248908,0.158416,0.303413,0.166887,1.034483,1.669173,0.976974,1.388145
5,6.929204,5.333333,5.272532,5.017699,0.324752,0.297174,0.338287,0.303442,0.436556,0.391512,0.452975,0.407115,0.255814,0.221757,0.243553,0.213135,1.646018,1.518519,1.551502,1.322124
6,3.306122,2.777143,4.127698,3.353503,0.320874,0.307641,0.337168,0.307965,0.409555,0.383105,0.428431,0.369275,0.236715,0.193133,0.229695,0.171893,1.306122,1.080000,1.327338,1.208599
9,4.661871,2.531250,3.545455,4.268382,0.304283,0.303559,0.279197,0.303196,0.387344,0.379114,0.391514,0.380435,0.191176,0.132159,0.243590,0.132732,1.489209,1.368750,1.189394,1.329044
10,4.155556,6.622222,5.714286,5.463492,0.314656,0.274168,0.311535,0.281162,0.425009,0.347073,0.426036,0.355036,0.203333,0.205000,0.191429,0.201429,1.416667,1.466667,1.528571,1.490476
11,5.894366,2.882022,4.287335,2.536204,0.318848,0.332573,0.297445,0.313433,0.422015,0.451846,0.393145,0.413086,0.248804,0.222707,0.227154,0.246291,1.500000,0.910112,1.378072,1.003914
12,3.566038,3.578313,4.385151,3.757455,0.292085,0.311905,0.298701,0.300718,0.391481,0.400292,0.435341,0.363988,0.239316,0.276316,0.213938,0.254412,1.433962,1.102410,1.308585,1.055666
13,2.618182,3.302158,3.320072,4.098672,0.313534,0.317174,0.305681,0.333632,0.382154,0.409795,0.386521,0.404595,0.346667,0.282723,0.321945,0.300539,1.109091,1.143885,1.177215,1.297913


In [66]:
Today_pred = model.predict(Todays_Games_to_predict)

In [67]:
Todays_Games_clean['Prediction'] = Today_pred

C:\Users\sanch\AppData\Local\Temp\ipykernel_24172\909423359.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Todays_Games_clean['Prediction'] = Today_pred


In [68]:
Todays_Games_clean.loc[Todays_Games_clean['Prediction'] == 1, 'Winner'] = Todays_Games_clean['Home_Team']
Todays_Games_clean.loc[Todays_Games_clean['Prediction'] == 0, 'Winner'] = Todays_Games_clean['Away_Team']

C:\Users\sanch\AppData\Local\Temp\ipykernel_24172\1681972152.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Todays_Games_clean.loc[Todays_Games_clean['Prediction'] == 1, 'Winner'] = Todays_Games_clean['Home_Team']


In [69]:
Todays_Games_clean['Winner']

0       D'backs
1       Yankees
2          Rays
5     Blue Jays
6          Mets
9     White Sox
10       Astros
11      Dodgers
12        Twins
13       Padres
14       Giants
Name: Winner, dtype: object

In [70]:
Todays_Games_clean

,Away_Team,Away_Team_short,Home_Team,Home_Team_short,Away_Pitcher,Home_Pitcher,Away_Pitcher_ID,Home_Pitcher_ID,ERA_10_h,ERA_10_v,ERA_35_h,ERA_35_v,OBP_162_h,OBP_162_v,OBP_30_h,OBP_30_v,SLG_162_h,SLG_162_v,SLG_30_h,SLG_30_v,SO_perc_10_h,SO_perc_10_v,SO_perc_35_h,SO_perc_35_v,WHIP_10_h,WHIP_10_v,WHIP_35_h,WHIP_35_v,Prediction,Winner
0,D'backs,ARI,Pirates,PIT,Zac Gallen,Johan Oviedo,gallz001,oviej001,3.126316,1.064039,4.286908,2.712919,0.285542,0.299966,0.301457,0.278704,0.363500,0.384069,0.376376,0.366633,0.240876,0.337553,0.202226,0.268054,1.263158,0.650246,1.454039,0.937799,0,D'backs
1,Yankees,NYA,Reds,CIN,Clarke Schmidt,Ben Lively,schmc002,liveb001,4.313609,4.222222,4.336364,4.419048,0.294582,0.318867,0.268501,0.327679,0.374025,0.425724,0.338477,0.435259,0.181070,0.235000,0.155650,0.208095,1.295858,1.566667,1.336364,1.528571,0,Yankees
2,Brewers,MIL,Rays,TBA,Adrian Houser,Shane McClanahan,housa001,mccls003,3.413793,6.496241,2.486842,3.820268,0.309027,0.308988,0.314932,0.318505,0.388920,0.410223,0.388177,0.424631,0.248908,0.158416,0.303413,0.166887,1.034483,1.669173,0.976974,1.388145,1,Rays
5,Orioles,BAL,Blue Jays,TOR,Kyle Gibson,Yusei Kikuchi,gibsk002,kikuy001,6.929204,5.333333,5.272532,5.017699,0.324752,0.297174,0.338287,0.303442,0.436556,0.391512,0.452975,0.407115,0.255814,0.221757,0.243553,0.213135,1.646018,1.518519,1.551502,1.322124,1,Blue Jays
6,Guardians,CLE,Mets,NYN,Cal Quantrill,Carlos Carrasco,quanc001,carrc003,3.306122,2.777143,4.127698,3.353503,0.320874,0.307641,0.337168,0.307965,0.409555,0.383105,0.428431,0.369275,0.236715,0.193133,0.229695,0.171893,1.306122,1.080000,1.327338,1.208599,1,Mets
9,Royals,KCA,White Sox,CHA,Zack Greinke,Michael Kopech,greiz001,kopem001,4.661871,2.531250,3.545455,4.268382,0.304283,0.303559,0.279197,0.303196,0.387344,0.379114,0.391514,0.380435,0.191176,0.132159,0.243590,0.132732,1.489209,1.368750,1.189394,1.329044,1,White Sox
10,Athletics,OAK,Astros,HOU,Ken Waldichuk,Brandon Bielak,waldk003,bielb001,4.155556,6.622222,5.714286,5.463492,0.314656,0.274168,0.311535,0.281162,0.425009,0.347073,0.426036,0.355036,0.203333,0.205000,0.191429,0.201429,1.416667,1.466667,1.528571,1.490476,1,Astros
11,Dodgers,LAN,Cardinals,SLN,Tony Gonsolin,Steven Matz,gonst001,matzs001,5.894366,2.882022,4.287335,2.536204,0.318848,0.332573,0.297445,0.313433,0.422015,0.451846,0.393145,0.413086,0.248804,0.222707,0.227154,0.246291,1.500000,0.910112,1.378072,1.003914,0,Dodgers
12,Twins,MIN,Angels,ANA,Joe Ryan,Reid Detmers,ryanj002,detmr001,3.566038,3.578313,4.385151,3.757455,0.292085,0.311905,0.298701,0.300718,0.391481,0.400292,0.435341,0.363988,0.239316,0.276316,0.213938,0.254412,1.433962,1.102410,1.308585,1.055666,0,Twins
13,Red Sox,BOS,Padres,SDN,James Paxton,Blake Snell,paxtj001,snelb001,2.618182,3.302158,3.320072,4.098672,0.313534,0.317174,0.305681,0.333632,0.382154,0.409795,0.386521,0.404595,0.346667,0.282723,0.321945,0.300539,1.109091,1.143885,1.177215,1.297913,1,Padres
